<a href="https://colab.research.google.com/github/milanochuang/518job_parser/blob/master/job_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import datetime
import json

In [2]:
url = "https://www.518.com.tw/job-index.html"
url_2 = "https://www.518.com.tw/job-index-P-{}.html?ai=0&ai=0"

In [3]:
def jobURL_grabber(URL, list):
    r = requests.get(URL)
    jobpage = BeautifulSoup(r.text, "html.parser")
    for i in jobpage.find_all('a', href=True):
        if 'https://www.518.com.tw/job' in i['href']:
          if 'index' not in i['href']: # 跳過目錄頁
            if i['href'] in list:
              continue
            else:
              list.append(i['href'])
    return list

In [4]:
pageIndex = 1
jobUrlLIST_1 = list()
jobUrlLIST_2 = list()
print("正在爬取第{}頁".format(pageIndex))
jobUrlLIST_1 = jobURL_grabber(url, jobUrlLIST_1)
for i in range(2, 3): #正式抓取應為151
    pageIndex += 1
    print("正在爬取第{}頁".format(pageIndex))
    jobUrlLIST_2 = jobURL_grabber("https://www.518.com.tw/job-index-P-{}.html?ai=0&ai=0".format(i), jobUrlLIST_2)

正在爬取第1頁
正在爬取第2頁


In [5]:
for i in jobUrlLIST_1:
  print(i)

https://www.518.com.tw/job-N6V3Xd.html
https://www.518.com.tw/job-Gj5rW0.html
https://www.518.com.tw/job-ynjpoB.html
https://www.518.com.tw/job-GRwEAw.html
https://www.518.com.tw/job-y3P2j1.html
https://www.518.com.tw/job-Lmj0EW.html
https://www.518.com.tw/job-L9PD1M.html
https://www.518.com.tw/job-GRBDMw.html
https://www.518.com.tw/job-ywEA44.html
https://www.518.com.tw/job-GK5g1R.html
https://www.518.com.tw/job-yOekqn.html
https://www.518.com.tw/job-ydeY0E.html
https://www.518.com.tw/job-LgjopO.html
https://www.518.com.tw/job-LgjV4v.html
https://www.518.com.tw/job-y2P7Jp.html
https://www.518.com.tw/job-yYY7kE.html
https://www.518.com.tw/job-ybbraE.html
https://www.518.com.tw/job-Lz5kxZ.html
https://www.518.com.tw/job-N6qYRK.html
https://www.518.com.tw/job-N6Zoe7.html
https://www.518.com.tw/job-GKemWE.html
https://www.518.com.tw/job-GxjJx2.html
https://www.518.com.tw/job-y5dg16.html
https://www.518.com.tw/job-yMmBd2.html
https://www.518.com.tw/job-Gjpo4A.html
https://www.518.com.tw/jo

In [6]:
for i in jobUrlLIST_2:
  print(i)

https://www.518.com.tw/job-GRP4Mg.html?pi=2
https://www.518.com.tw/job-NPoDEQ.html?pi=2
https://www.518.com.tw/job-GQORP6.html?pi=2
https://www.518.com.tw/job-Lz5Y0q.html?pi=2
https://www.518.com.tw/job-Gajn2J.html?pi=2
https://www.518.com.tw/job-ybdolv.html?pi=2
https://www.518.com.tw/job-G84KQ5.html?pi=2
https://www.518.com.tw/job-N6axE1.html?pi=2
https://www.518.com.tw/job-yw6dW4.html?pi=2
https://www.518.com.tw/job-GpMV3A.html?pi=2
https://www.518.com.tw/job-GxRkYa.html?pi=2
https://www.518.com.tw/job-LDeO0b.html?pi=2
https://www.518.com.tw/job-GKejJV.html?pi=2
https://www.518.com.tw/job-NqjM3K.html?pi=2
https://www.518.com.tw/job-GojgoD.html?pi=2
https://www.518.com.tw/job-Goj9qD.html?pi=2
https://www.518.com.tw/job-ywAwxe.html?pi=2
https://www.518.com.tw/job-ybjr9X.html?pi=2
https://www.518.com.tw/job-ywY6D4.html?pi=2
https://www.518.com.tw/job-yB1oOq.html?pi=2
https://www.518.com.tw/job-LgAEEW.html?pi=2
https://www.518.com.tw/job-y347Be.html?pi=2
https://www.518.com.tw/job-LzdlD

In [7]:
jobUrlLIST = jobUrlLIST_1 + jobUrlLIST_2

In [8]:
def data_cleaner(string):
  string = [x.replace("\r","") for x in string]
  joblist = [x.replace("\n","") for x in string]
  return joblist

In [9]:
def jobLIST_grabber(jobUrl):
  resultDICT = {
        "jobTitle": "",
        "company": "",
        "salary":[''],
        "geoInfo":{"address": ""},
        "majoring":[''],
        "tool":[''],
        "skills":[''],
        "certificate":[''],
        "experience":[''],
        "license":'',
        "language":'',
        "dialect":''
    }
  r = requests.get(jobUrl)
  jobpage = BeautifulSoup(r.text, "html.parser")
  title = data_cleaner(jobpage.find(class_ = "job-title"))[0]
  resultDICT['jobTitle'] = title
  salary = jobpage.find(class_ = 'jobItem').find("li").find("span").text[1:]
  resultDICT['salary'] = salary
  company = jobpage.select("a span")[1].text
  resultDICT['company'] = company
  address = jobpage.find(class_ = 'jobItem').find_all("li")[1].find("span").text[1:]
  resultDICT['geoInfo']['address'] = address
  condition = jobpage.find(class_ = 'condition').find_all('li')
  index = 1
  for i in condition:
    if i.find("span") != None:
      # print("第{}個".format(index), i.text[0:4], i.find("span").text)
      index+=1
      if '工作經驗' in i.text:
        if i.text[0:4] == "工作經驗":
          experience = i.find("span").text[1:]
          resultDICT['experience'] = experience
      else:
        experience = ''
      if '科系限制' in i.text:
        if i.text[0:4] == "科系限制":
          majoring = i.find("span").text
          resultDICT['majoring'] = majoring
      else:
        majoring = ''
      if '語文條件' in i.text:
        if i.text[0:4] == "語文條件":
          language = i.find("span").text[1:].partition("方")[0]
          language = language.replace("\xa0", " ")
          resultDICT['language'] = language
      else:
        language = ''
      if '方言條件' in i.text:
        if i.text[0:4] == "方言條件":
          dialect = i.find("span").text
          dialect = dialect.replace("\xa0", " ")
          resultDICT['dialect'] = dialect
      else:
        dialect = ''
      if '擅長工具' in i.text:
        if i.text[0:4] == "擅長工具":
          tool = i.find("span").text[1:]
          resultDICT['tool'] = tool
      else:
        tool = ''
      if '工作技能' in i.text:
        if i.text[0:4] == "工作技能":
          if '駕照' in i.find("span").text:
            skills = i.find("span").text[1:].partition("需")[0]
            resultDICT['skills'] = skills
          elif '證照' in i.find("span").text:
            skills = i.find("span").text[1:].partition("所")[0]
            resultDICT['skills'] = skills
          else:
            skills = i.find("span").text[1:]
            resultDICT['skills'] = skills
      else:
        skills = ''
      if '所需證照' in i.text:
        if i.text[0:4] == "所需證照":
          certificate = i.find("span").text
          resultDICT['certificate'] = certificate
      else:
        certificate = ''
      if '需備駕照' in i.text:
        if i.text[0:4] == "需備駕照":
          license = i.find("span").text
          resultDICT['license'] = license
      else:
        license = ''
  return resultDICT

In [10]:
resultLIST = list()
for jobUrl in jobUrlLIST:
  if jobUrlLIST.index(jobUrl) == int(len(jobUrlLIST)*0.25):
    print("已爬取25%的網頁")
  if jobUrlLIST.index(jobUrl) == int(len(jobUrlLIST)*0.5):
    print("已爬取50%的網頁")
  if jobUrlLIST.index(jobUrl) == int(len(jobUrlLIST)*0.75):
    print("已爬取75%的網頁")
  resultDICT = jobLIST_grabber(jobUrl)
  resultLIST.append(resultDICT)
print("爬取完畢")

已爬取25%的網頁
已爬取50%的網頁
已爬取75%的網頁
爬取完畢


In [11]:
len(resultLIST)

66

In [12]:
for i in resultLIST:
  print(i)

{'jobTitle': '作業員', 'company': '瑞瓏紙器有限公司', 'salary': '月薪 24,000 元', 'geoInfo': {'address': '彰化縣福興鄉福三路三段197巷66號'}, 'majoring': [''], 'tool': '不拘', 'skills': '不拘', 'certificate': [''], 'experience': '無經驗可', 'license': '', 'language': '不拘', 'dialect': ''}
{'jobTitle': '行政助理', 'company': '瑞瓏紙器有限公司', 'salary': '月薪 26,000 元', 'geoInfo': {'address': '彰化縣福興鄉福三路三段197巷66號'}, 'majoring': [''], 'tool': '不拘', 'skills': '不拘', 'certificate': [''], 'experience': '無經驗可', 'license': '', 'language': '不拘', 'dialect': ''}
{'jobTitle': 'PT晚班房務', 'company': '花鄉旅館連鎖巨蛋店(歐洲旅舘)', 'salary': '時薪 160 元以上', 'geoInfo': {'address': '高雄市左營區文智路95號'}, 'majoring': [''], 'tool': '不拘', 'skills': '不拘', 'certificate': [''], 'experience': '不拘', 'license': '', 'language': '不拘', 'dialect': ''}
{'jobTitle': '活動企劃專員', 'company': '京采活動工程股份有限公司', 'salary': '月薪 24,000 至 45,000 元', 'geoInfo': {'address': '桃園區吉安一街１１８號'}, 'majoring': [''], 'tool': 'PowerPoint、、、Excel、Adobe Acrobat、、、Word', 'skills': '公關活動規劃與執行、公關議題與危機處理、活動主持與場控、記者會規劃與執行

In [13]:
with open("518JobLIST_{}.json".format(datetime.date.today()), "w", encoding="utf-8") as f:
        json.dump(resultLIST, f, ensure_ascii=False)